In [ ]:
pip install rioxarray

In [ ]:
pip install numpy gdal==$(gdal-config --version)

In [ ]:
pip install scipy

In [ ]:
pip install matplotlib

In [ ]:
pip install geopandas

In [6]:
from osgeo import gdal
import xarray as xr
import rioxarray as rxr
import os
from glob import glob
from datetime import datetime, timedelta
import numpy as np
import calendar
import numpy as n
import rasterio

In [11]:
import os
from datetime import datetime, timedelta
from glob import glob
import numpy as np
from osgeo import gdal, osr
import warnings
warnings.filterwarnings('ignore')

def get_delhi_bounds():
    """Define the bounding box for Delhi region"""
    return {
        'min_lon': 76.5,  # Western boundary of Delhi
        'max_lon': 77.5,  # Eastern boundary of Delhi
        'min_lat': 28.3,  # Southern boundary of Delhi
        'max_lat': 28.9   # Northern boundary of Delhi
    }

def convert_grd_to_daily_tiff(grd_file, output_dir):
    """Convert GRD file to daily GeoTIFF files for Delhi region"""
    try:
        # Extract year from filename (assuming format YYYY.grd)
        year = os.path.basename(grd_file).split('.')[0]
        print(f"Processing year: {year}")
        
        # Open the .grd file and read its content
        with open(grd_file, 'rb') as f:
            data = f.read()
        print(f"Read {len(data)} bytes from {grd_file}")
        
        # Determine the number of days in the year
        is_leap_year = (int(year) % 4 == 0 and int(year) % 100 != 0) or (int(year) % 400 == 0)
        days_in_year = 366 if is_leap_year else 365
        print(f"Year {year} has {days_in_year} days")
        
        # Convert binary data to numpy array
        all_data = np.frombuffer(data, dtype=np.float32)
        
        # For 0.25° x 0.25° grid covering India
        grid_size = 129 * 135  # Total number of grid cells
        
        # Get Delhi bounds
        bounds = get_delhi_bounds()
        
        # Calculate array indices for clipping
        min_x = int((bounds['min_lon'] - 65.0) / 0.25)
        max_x = int((bounds['max_lon'] - 65.0) / 0.25)
        min_y = int((40.0 - bounds['max_lat']) / 0.25)  # Flip Y coordinates
        max_y = int((40.0 - bounds['min_lat']) / 0.25)
        
        # Calculate new dimensions
        new_width = max_x - min_x + 1
        new_height = max_y - min_y + 1
        
        for day in range(days_in_year):
            date = datetime(int(year), 1, 1) + timedelta(days=day)
            output_filename = f"imd_25_delhi_{date.strftime('%Y-%m-%d')}.tif"
            output_file = os.path.join(output_dir, output_filename)
            
            # Extract data for the current day and the Delhi region
            day_data = all_data[day * grid_size : (day + 1) * grid_size].reshape(129, 135)[min_y:max_y+1, min_x:max_x+1]
            
            # Create a new GeoTIFF file
            driver = gdal.GetDriverByName('GTiff')
            dataset = driver.Create(output_file, new_width, new_height, 1, gdal.GDT_Float32)
            
            # Set geotransform for the clipped region
            geotransform = (
                bounds['min_lon'],  # Left edge
                0.25,               # Pixel width
                0,                  # Rotation (0 if image is "north up")
                bounds['max_lat'],  # Top edge
                0,                  # Rotation (0 if image is "north up")
                -0.25              # Pixel height (negative as it goes from top to bottom)
            )
            dataset.SetGeoTransform(geotransform)
            
            # Set projection (WGS84)
            srs = osr.SpatialReference()
            srs.ImportFromEPSG(4326)
            dataset.SetProjection(srs.ExportToWkt())
            
            # Write the clipped data
            dataset.GetRasterBand(1).WriteArray(day_data)
            
            # Set NoData value
            dataset.GetRasterBand(1).SetNoDataValue(-999)
            
            # Close the dataset
            dataset = None
            
            if os.path.exists(output_file):
                print(f"Successfully saved {output_file}")
            else:
                print(f"Failed to create {output_file}")
            
    except Exception as e:
        print(f"Failed to process {grd_file}: {str(e)}")
        raise

def main():
    # Define the directory containing the rainfall .grd files
    rainfall_dir = r'/home/stormej/dev/varsha/data/rain/rain_grd'
    
    # Define the output directory for the daily .tif files
    output_dir = r'/home/stormej/dev/varsha/data/rain/rain_tif'
    os.makedirs(output_dir, exist_ok=True)
    
    # List all .grd files in the rainfall directory
    grd_files = glob(os.path.join(rainfall_dir, '*.grd'))
    
    if not grd_files:
        print(f"No .grd files found in {rainfall_dir}")
    else:
        for grd_file in grd_files:
            print(f"Processing {grd_file}...")
            convert_grd_to_daily_tiff(grd_file, output_dir)
    
    print("Script execution completed.")
main()

Processing /home/stormej/dev/varsha/data/rain/rain_grd/2000.grd...
Processing year: 2000
Read 25495560 bytes from /home/stormej/dev/varsha/data/rain/rain_grd/2000.grd
Year 2000 has 366 days
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif/imd_25_delhi_2000-01-01.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif/imd_25_delhi_2000-01-02.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif/imd_25_delhi_2000-01-03.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif/imd_25_delhi_2000-01-04.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif/imd_25_delhi_2000-01-05.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif/imd_25_delhi_2000-01-06.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif/imd_25_delhi_2000-01-07.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif/imd_25_delhi_2000-01-08.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif/imd_25_delhi_2000-0

In [14]:
import os
from datetime import datetime, timedelta
from glob import glob
import numpy as np
from osgeo import gdal, osr
import warnings
warnings.filterwarnings('ignore')

def get_delhi_bounds():
    """Define the bounding box for Delhi region"""
    return {
        'min_lon': 76.5,  # Western boundary of Delhi
        'max_lon': 77.5,  # Eastern boundary of Delhi
        'min_lat': 28.3,  # Southern boundary of Delhi
        'max_lat': 28.9   # Northern boundary of Delhi
    }

def convert_grd_to_monthly_tiff(grd_file, output_dir):
    """Convert GRD file to monthly GeoTIFF files for Delhi region"""
    try:
        # Extract year from filename (assuming format YYYY.grd)
        year = os.path.basename(grd_file).split('.')[0]
        print(f"Processing year: {year}")
        
        # Open the .grd file and read its content
        with open(grd_file, 'rb') as f:
            data = f.read()
        print(f"Read {len(data)} bytes from {grd_file}")
        
        # Determine the number of days in the year
        is_leap_year = (int(year) % 4 == 0 and int(year) % 100 != 0) or (int(year) % 400 == 0)
        days_in_year = 366 if is_leap_year else 365
        print(f"Year {year} has {days_in_year} days")
        
        # Convert binary data to numpy array
        all_data = np.frombuffer(data, dtype=np.float32)
        
        # For 0.25° x 0.25° grid covering India
        grid_size = 129 * 135  # Total number of grid cells
        
        # Get Delhi bounds
        bounds = get_delhi_bounds()
        
        # Calculate array indices for clipping
        min_x = int((bounds['min_lon'] - 65.0) / 0.25)
        max_x = int((bounds['max_lon'] - 65.0) / 0.25)
        min_y = int((40.0 - bounds['max_lat']) / 0.25)  # Flip Y coordinates
        max_y = int((40.0 - bounds['min_lat']) / 0.25)
        
        # Calculate new dimensions
        new_width = max_x - min_x + 1
        new_height = max_y - min_y + 1
        
        # Create a 3D numpy array to store monthly rainfall data
        monthly_rainfall = np.zeros((12, new_height, new_width), dtype=np.float32)
        
        for day in range(days_in_year):
            date = datetime(int(year), 1, 1) + timedelta(days=day)
            month = date.month - 1  # Adjust month index to 0-11 range
            
            # Extract data for the current day and the Delhi region
            day_data = all_data[day * grid_size : (day + 1) * grid_size].reshape(129, 135)[min_y:max_y+1, min_x:max_x+1]
            
            # Add the daily rainfall to the corresponding monthly array
            monthly_rainfall[month] += day_data
        
        for month in range(12):
            date = datetime(int(year), month + 1, 1)
            output_filename = f"imd_25_delhi_{date.strftime('%Y-%m')}.tif"
            output_file = os.path.join(output_dir, output_filename)
            
            # Create a new GeoTIFF file
            driver = gdal.GetDriverByName('GTiff')
            dataset = driver.Create(output_file, new_width, new_height, 1, gdal.GDT_Float32)
            
            # Set geotransform for the clipped region
            geotransform = (
                bounds['min_lon'],  # Left edge
                0.25,               # Pixel width
                0,                  # Rotation (0 if image is "north up")
                bounds['max_lat'],  # Top edge
                0,                  # Rotation (0 if image is "north up")
                -0.25              # Pixel height (negative as it goes from top to bottom)
            )
            dataset.SetGeoTransform(geotransform)
            
            # Set projection (WGS84)
            srs = osr.SpatialReference()
            srs.ImportFromEPSG(4326)
            dataset.SetProjection(srs.ExportToWkt())
            
            # Write the monthly rainfall data
            dataset.GetRasterBand(1).WriteArray(monthly_rainfall[month])
            
            # Set NoData value
            dataset.GetRasterBand(1).SetNoDataValue(-999)
            
            # Close the dataset
            dataset = None
            
            if os.path.exists(output_file):
                print(f"Successfully saved {output_file}")
            else:
                print(f"Failed to create {output_file}")
            
    except Exception as e:
        print(f"Failed to process {grd_file}: {str(e)}")
        raise

def main():
    # Define the directory containing the rainfall .grd files
    rainfall_dir = r'/home/stormej/dev/varsha/data/rain/rain_grd'
    
    # Define the output directory for the monthly .tif files
    output_dir = r'/home/stormej/dev/varsha/data/rain/rain_tif_monthly'
    os.makedirs(output_dir, exist_ok=True)
    
    # List all .grd files in the rainfall directory
    grd_files = glob(os.path.join(rainfall_dir, '*.grd'))
    
    if not grd_files:
        print(f"No .grd files found in {rainfall_dir}")
    else:
        for grd_file in grd_files:
            print(f"Processing {grd_file}...")
            convert_grd_to_monthly_tiff(grd_file, output_dir)
    
    print("Script execution completed.")


main()

Processing /home/stormej/dev/varsha/data/rain/rain_grd/2000.grd...
Processing year: 2000
Read 25495560 bytes from /home/stormej/dev/varsha/data/rain/rain_grd/2000.grd
Year 2000 has 366 days
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif_monthly/imd_25_delhi_2000-01.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif_monthly/imd_25_delhi_2000-02.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif_monthly/imd_25_delhi_2000-03.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif_monthly/imd_25_delhi_2000-04.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif_monthly/imd_25_delhi_2000-05.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif_monthly/imd_25_delhi_2000-06.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif_monthly/imd_25_delhi_2000-07.tif
Successfully saved /home/stormej/dev/varsha/data/rain/rain_tif_monthly/imd_25_delhi_2000-08.tif
Successfully saved /home/stormej/dev/varsh

In [4]:
#Resampling imd monthly data to imd data i.e 0.25 degree

import os
import rasterio
from rasterio.enums import Resampling
# # Paths
# rainfall_folder = r'/home/stormej/dev/btech-project/data/rain_tif'  # Folder containing all rainfall files
rainfall_folder = r'/home/stormej/dev/varsha/data/reference_tif'  # Folder containing reference files
input_folder = r'/home/stormej/dev/varsha/data/rain/rain_tif_monthly'  # Folder containing all LST files
output_folder = r'/home/stormej/dev/varsha/data/rain/rain_resampled_monthly'  # Folder to save resampled files

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


# Find a reference rainfall file
reference_rainfall_path = None
for filename in os.listdir(rainfall_folder):
    if filename.endswith('.tif'):
        reference_rainfall_path = os.path.join(rainfall_folder, filename)
        break

if reference_rainfall_path is None:
    raise FileNotFoundError("No TIF file found in the rainfall folder")

print(f"Using reference rainfall file: {reference_rainfall_path}")

# Open the reference rainfall file to get its properties
with rasterio.open(reference_rainfall_path) as rainfall_src:
    rainfall_shape = (rainfall_src.height, rainfall_src.width)
    rainfall_crs = rainfall_src.crs
    rainfall_transform = rainfall_src.transform

print(f"Reference rainfall data shape: {rainfall_shape}, CRS: {rainfall_crs}")

# Process all LST files in the folder
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        lst_path = os.path.join(input_folder, filename)
        print(f"\nProcessing: {filename}")
        
        with rasterio.open(lst_path) as lst_src:
            lst_data = lst_src.read(1)
            print(f"LST data shape: {lst_data.shape}, dtype: {lst_data.dtype}")
            
            # Resample LST data using average method
            resampled_lst = lst_src.read(
                out_shape=(1, rainfall_shape[0], rainfall_shape[1]),
                resampling=Resampling.average
            )
            print(f"Resampled LST shape: {resampled_lst.shape}, dtype: {resampled_lst.dtype}")
            
            # Extract date from LST filename (handling format: lst_south_2010-01-01.tif)
            # Split by underscore and get the last part before .tif
            date_part = filename.split('_')[-1].split('.')[0]
            
            # Create output filename
            output_filename = f'imd_rain_sum_resampled_{date_part}.tif'
            output_path = os.path.join(output_folder, output_filename)
            
            # Save the resampled LST
            with rasterio.open(
                output_path,
                'w',
                driver='GTiff',
                height=resampled_lst.shape[1],
                width=resampled_lst.shape[2],
                count=1,
                dtype=resampled_lst.dtype,
                crs=rainfall_crs,
                transform=rainfall_transform,
            ) as dst:
                dst.write(resampled_lst[0], 1)
        
        print(f"Resampled LST saved to: {output_path}")

print("\nAll LST files have been resampled.")

Using reference rainfall file: /home/stormej/dev/varsha/data/reference_tif/refrence_tif.tif
Reference rainfall data shape: (129, 135), CRS: EPSG:4326

Processing: imd_25_delhi_2000-01.tif
LST data shape: (3, 5), dtype: float32
Resampled LST shape: (1, 129, 135), dtype: float32
Resampled LST saved to: /home/stormej/dev/varsha/data/rain/rain_resampled_monthly/imd_rain_sum_resampled_2000-01.tif

Processing: imd_25_delhi_2000-02.tif
LST data shape: (3, 5), dtype: float32
Resampled LST shape: (1, 129, 135), dtype: float32
Resampled LST saved to: /home/stormej/dev/varsha/data/rain/rain_resampled_monthly/imd_rain_sum_resampled_2000-02.tif

Processing: imd_25_delhi_2000-03.tif
LST data shape: (3, 5), dtype: float32
Resampled LST shape: (1, 129, 135), dtype: float32
Resampled LST saved to: /home/stormej/dev/varsha/data/rain/rain_resampled_monthly/imd_rain_sum_resampled_2000-03.tif

Processing: imd_25_delhi_2000-04.tif
LST data shape: (3, 5), dtype: float32
Resampled LST shape: (1, 129, 135), dt